In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

In [ ]:
yerevan = gpd.read_file("Yerevan shapefile/Yerevan.shp")
yerevan.head()

In [ ]:
yerevan.plot()

In [ ]:
yerevan.crs

In [ ]:
# load yerevan city from openstreetmap using osmmnx api
city = ox.geocode_to_gdf("Yerevan, Armenia")
print(city.crs)
city = ox.project_gdf(city)
fig, ax = ox.plot_footprints(city)
print(city.crs)

In [ ]:
city.to_file('Output/Yerevan_city.shp')

In [ ]:
geometry = city['geometry'].iloc[0]
print(type(geometry))

In [ ]:
geometry_cut = ox.utils_geo._quadrat_cut_geometry(geometry, quadrat_width = 750)
print(type(geometry_cut))

In [ ]:
polylist = [p for p in geometry_cut]

# plot city
west, south, east, north = city.unary_union.bounds

fig, ax = plt.subplots(figsize=(40,40))
for polygon, n in zip(geometry_cut, np.arange(len(polylist))):
    p = polygon.representative_point().coords[:][0]
    patch = PolygonPatch(polygon, fc='#ffffff', ec='#000000', alpha=0.5, zorder=2)
    ax.add_patch(patch)
    plt.annotate(text=n, xy=p, horizontalalignment='center', size=15)
    
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
polyframe = gpd.GeoDataFrame(geometry=polylist)
polyframe.crs = city.crs
print(polyframe.crs)
polyframe.head()

In [ ]:
len(polyframe)

In [ ]:
import contextily as ctx
# ctx uses epsg:3857
polyframe_3857 = polyframe.to_crs(epsg=3857)
west, south, east, north = polyframe_3857.unary_union.bounds

ax = polyframe_3857.plot(figsize=(40,40), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, zoom=13)
ax.set_xlim(west, east)
ax.set_ylim(south, north)

In [ ]:
l = len(polylist)
OD_matrix = np.zeros(shape=(l,l))
print(OD_matrix)

In [ ]:
import pickle
pkl_file = open("Materials/Yerevan_grid_population.pkl", "rb")
yerevan_pop = pickle.load(pkl_file)
pkl_file.close()
print(yerevan_pop)

In [ ]:
plt.hist(yerevan_pop, bins=50)
plt.show()

In [ ]:
yerevan_gdf = gpd.read_file('Yerevan grid shapefile/Yerevan.shp')
yerevan_gdf.head()

In [ ]:
print('Yerevan geodataframe length: ', len(yerevan_gdf))
print('Yerevan population array length: ', len(yerevan_pop))

In [ ]:
yerevan_gdf['population'] = yerevan_pop
yerevan_gdf.head()

In [ ]:
polylist = [p for p in polyframe_3857.geometry]

# plot city
west, south, east, north = polyframe_3857.unary_union.bounds

ax = polyframe_3857.plot(figsize=(40,40), alpha=0.5, edgecolor='k', zorder=5)
for polygon, n in zip(polylist, np.arange(len(polylist))):
    p = polygon.representative_point().coords[:][0]
    patch = PolygonPatch(polygon, fc='#ffffff', ec='#000000', alpha=0.5, zorder=2)
    ax.add_patch(patch)
    plt.annotate(text=n, xy=p, horizontalalignment='center', size=15)
ctx.add_basemap(ax, zoom=13)
    
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
plt.rcParams.update({'font.size':32})
west, south, east, north = yerevan_gdf.unary_union.bounds

fig, ax = plt.subplots(figsize=(40,40))
yerevan_gdf.plot(ax=ax, column = 'population', legend=False, cmap='rainbow')

cbax = fig.add_axes([0.915, 0.175, 0.02, 0.7])

sm = plt.cm.ScalarMappable(cmap='rainbow', \
                          norm = plt.Normalize(vmin=min(yerevan_gdf.population), vmax=max(yerevan_gdf.population)))

sm._A = []

# draw colormap into cbax

fig.colorbar(sm, cax=cbax, format="%d")

ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()